Data collection ▶ we created a google form and manually Havested datasets from Twitter, Facebook, Instagram and other Social Media Platforms where Nigerian Multilingual comments can be found.

Form ▶  https://docs.google.com/spreadsheets/d/1lcxmL3Dh7g3Tr1FIb6xVfBkJ8KuySMUwLbrZVX4XOZQ/edit?resourcekey=&gid=1995333278#gid=1995333278

Data Pre-Processing (we can't feed textual data directly to our ML model coz it only understands numerical data) ▶

Train Test Split (training dataset would be used to train the model while testing dataset would be used to evaluate our model) ▶

Logistic Regression Model (as this is a classification problem coz we're classifying the Tweets into Positive{+} Tweets , Neutral{0} or Negative{-} Tweets) ▶▶▶▶▶▶▶

New Tweet-data➰Trained Logistic Regression Model ➰ {+} or {-} prediction

**Importing the Dependencies**

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

**Upload your Dataset Excel File to Pandas**

In [2]:
#MultiLang_Data = pd.ExcelFile('/content/DATASET FOR LOGISTIC REGRESSION SENTIMENT ANALYSIS.xlsx')

In [2]:
MultiLang_Data = pd.ExcelFile('/content/MAIN DATASET FOR LOGISTIC REGRESSION SENTIMENT ANALYSIS.xlsx')

In [3]:
MultiLang_Data.sheet_names

['GENERAL', 'PIDGIN', 'HAUSA', 'IGBO', 'YORUBA']

In [4]:
Hausa_Data = pd.read_excel(MultiLang_Data, 'HAUSA')
Igbo_Data = pd.read_excel(MultiLang_Data, 'IGBO')
Yoruba_Data = pd.read_excel(MultiLang_Data, 'YORUBA')
Pidgin_Data = pd.read_excel(MultiLang_Data, 'PIDGIN')
General_Data = pd.read_excel(MultiLang_Data, 'GENERAL')

## **DATA PREPROCESSING**

In [6]:


# Drop unnamed columns
Hausa_Data = Hausa_Data.drop(Hausa_Data.columns[Hausa_Data.columns.str.contains('unnamed',case = False)],axis = 1)
Igbo_Data = Igbo_Data.drop(Igbo_Data.columns[Igbo_Data.columns.str.contains('unnamed',case = False)],axis = 1)
Yoruba_Data = Yoruba_Data.drop(Yoruba_Data.columns[Yoruba_Data.columns.str.contains('unnamed',case = False)],axis = 1)
Pidgin_Data = Pidgin_Data.drop(Pidgin_Data.columns[Pidgin_Data.columns.str.contains('unnamed',case = False)],axis = 1)
General_Data = General_Data.drop(General_Data.columns[General_Data.columns.str.contains('unnamed',case = False)],axis = 1)


In [7]:
print(Hausa_Data.shape)
print(Igbo_Data.shape)
print(Yoruba_Data.shape)
print(Pidgin_Data.shape)

(1998, 3)
(627, 3)
(1292, 3)
(683, 3)


In [12]:

print(Hausa_Data.shape)
print(Igbo_Data.shape)
print(Yoruba_Data.shape)
print(Pidgin_Data.shape)
print(Hausa_Data.head(), '\n\n')
print(Igbo_Data.head())
print(Yoruba_Data.head())
print(Pidgin_Data.head())
print(General_Data.head())


(1998, 4)
(627, 3)
(1292, 3)
(683, 4)
                                           Sentence   \
0  Jagoran jam'iyyar Economic Freedom Fighters wa...   
1                    It shall not happen insha Allah   
2  Saniyar da ba ta da jela, Allah ke kore mata ƙ...   
3  Congratulations Brotherly....as u have taken t...   
4  Alhamdullah! Alhamdullah! I am incredibly grat...   

  What combination does the sentence have? sentiment  \
0                        English and Hausa   neutral   
1                        English and Hausa  positive   
2                        English and Hausa   neutral   
3                        English and Hausa  positive   
4                        English and Hausa  positive   

                                            Sentence  
0  jagoran jam'iyyar economic freedom fighters wa...  
1                    it shall not happen insha allah  
2  saniyar da ba ta da jela, allah ke kore mata ƙ...  
3  congratulations brotherly....as u have taken t...  
4  alhamdulla

In [13]:
print('\n For Hausa Sentences :')
for sentiments in Hausa_Data['sentiment'].unique():
  print( sentiments, (Hausa_Data['sentiment'] == sentiments).sum())


print('\n For Igbo Sentences :')
for sentiments in Igbo_Data['sentiment'].unique():
  print( sentiments, (Igbo_Data['sentiment'] == sentiments).sum())

print('\n For Pidgin Sentences :')
for sentiments in Pidgin_Data['sentiment'].unique():
  print( sentiments, (Pidgin_Data['sentiment'] == sentiments).sum())

print('\n For Yoruba Sentences :')
for sentiments in Yoruba_Data['sentiment'].unique():
  print( sentiments, (Yoruba_Data['sentiment'] == sentiments).sum())

print('\n For General Sentences :')
for sentiments in General_Data['sentiment'].unique():
  print( sentiments, (General_Data['sentiment'] == sentiments).sum())




 For Hausa Sentences :
neutral 541
positive 992
negative 462
nan 0

 For Igbo Sentences :
positive 111
negative 123
neutral 393

 For Pidgin Sentences :
neutral 364
negative 205
positive 114

 For Yoruba Sentences :
neutral 1141
negative 51
positive 100

 For General Sentences :
nan 0
positive 197
negative 211
neutral 245


In [11]:
Hausa_Data['sentiment'] = Hausa_Data['sentiment'].replace(' POSITIVE', 'positive')
Hausa_Data['sentiment'] = Hausa_Data['sentiment'].replace(' NEGATIVE', 'negative')
Hausa_Data['sentiment'] = Hausa_Data['sentiment'].replace(' NEUTRAL', 'neutral')
Hausa_Data['sentiment'] = Hausa_Data['sentiment'].replace('neutal', 'neutral')
Hausa_Data['Sentence'] = Hausa_Data['Sentence '].str.lower()

Pidgin_Data['sentiment'] = Pidgin_Data['sentiment'].replace(' POSITIVE', 'positive')
Pidgin_Data['sentiment'] = Pidgin_Data['sentiment'].replace(' NEGATIVE', 'negative')
Pidgin_Data['sentiment'] = Pidgin_Data['sentiment'].replace(' NEUTRAL', 'neutral')
Pidgin_Data['Sentence'] = Pidgin_Data['Sentence '].str.lower()

Igbo_Data['sentiment'] = Igbo_Data['sentiment'].replace(' POSITIVE', 'positive')
Igbo_Data['sentiment'] = Igbo_Data['sentiment'].replace(' NEGATIVE', 'negative')
Igbo_Data['sentiment'] = Igbo_Data['sentiment'].replace(' NEUTRAL', 'neutral')

Yoruba_Data['sentiment'] = Yoruba_Data['sentiment'].replace(' POSITIVE', 'positive')
Yoruba_Data['sentiment'] = Yoruba_Data['sentiment'].replace('positve', 'positive')
Yoruba_Data['sentiment'] = Yoruba_Data['sentiment'].replace(' NEGATIVE', 'negative')
Yoruba_Data['sentiment'] = Yoruba_Data['sentiment'].replace(' NEUTRAL', 'neutral')

In [34]:
print('General_Data\n',General_Data.isnull().sum())
print('\nHausa_Data\n',Hausa_Data.isnull().sum())
print('\nIgbo_Data\n',Igbo_Data.isnull().sum())
print('\nYoruba_Data\n',Yoruba_Data.isnull().sum())
print('\nPidgin_Data\n',Pidgin_Data.isnull().sum())

General_Data
 Sentence                                    0
What combination does the sentence have?    0
sentiment                                   0
dtype: int64

Hausa_Data
 What combination does the sentence have?    0
sentiment                                   0
Sentence                                    0
dtype: int64

Igbo_Data
 Sentence                                    0
What combination does the sentence have?    0
sentiment                                   0
dtype: int64

Yoruba_Data
 Sentence                                    0
What combination does the sentence have?    0
sentiment                                   0
dtype: int64

Pidgin_Data
 Sentence                                    0
What combination does the sentence have?    0
sentiment                                   0
Sentence                                    0
dtype: int64


In [29]:
Hausa_Data.head()

,What combination does the sentence have?,sentiment,Sentence
0,English and Hausa,neutral,Jagoran jam'iyyar Economic Freedom Fighters wa...
1,English and Hausa,positive,It shall not happen insha Allah it shall not h...
2,English and Hausa,neutral,"Saniyar da ba ta da jela, Allah ke kore mata ƙ..."
3,English and Hausa,positive,Congratulations Brotherly....as u have taken t...
4,English and Hausa,positive,Alhamdullah! Alhamdullah! I am incredibly grat...


In [32]:
Hausa_Data = Hausa_Data.fillna('Hausa')
Igbo_Data = Igbo_Data.fillna('Igbo')
Yoruba_Data = Yoruba_Data.fillna('Yoruba')
Pidgin_Data = Pidgin_Data.fillna('Pidgin')

In [36]:
Hausa_Data = Hausa_Data[Hausa_Data['sentiment'] != 'Hausa']

In [25]:
# Drop the original columns if you don't need them anymore
Hausa_Data = Hausa_Data.drop(["sentence1"], axis=1)

# Display the updated DataFrame
print(Hausa_Data.head())


  What combination does the sentence have? sentiment  \
0                        English and Hausa   neutral   
1                        English and Hausa  positive   
2                        English and Hausa   neutral   
3                        English and Hausa  positive   
4                        English and Hausa  positive   

                                            Sentence  
0  Jagoran jam'iyyar Economic Freedom Fighters wa...  
1  It shall not happen insha Allah it shall not h...  
2  Saniyar da ba ta da jela, Allah ke kore mata ƙ...  
3  Congratulations Brotherly....as u have taken t...  
4  Alhamdullah! Alhamdullah! I am incredibly grat...  


In [6]:
General_Data.dropna(inplace=True)
print('General_Data\n',General_Data.isnull().sum())
Hausa_Data.dropna(inplace=True)
print('\nHausa_Data\n',Hausa_Data.isnull().sum())

General_Data
 Sentence     0
sentiment    0
dtype: int64

Hausa_Data
 Sentence     0
sentiment    0
dtype: int64


In [37]:
print(Hausa_Data['sentiment'].value_counts())
print(Pidgin_Data['sentiment'].value_counts())
print(Igbo_Data['sentiment'].value_counts())
print(Yoruba_Data['sentiment'].value_counts())
print(General_Data['sentiment'].value_counts())

sentiment
positive    992
neutral     541
negative    462
Name: count, dtype: int64
sentiment
neutral     364
negative    205
positive    114
Name: count, dtype: int64
sentiment
neutral     393
negative    123
positive    111
Name: count, dtype: int64
sentiment
neutral     1141
positive     100
negative      51
Name: count, dtype: int64
sentiment
neutral     245
negative    211
positive    196
Name: count, dtype: int64


# **INDIVIDUAL ANALYTICS FOR EACH LANGUAGE USING THE FOUR MACHINE LEARNING ALGORITHMS**

### FOR **NIGERIAN-PIDGIN**

In [20]:
Pidgin_Data.head()

,Sentence,sentiment,Sentence
0,"Na new style of hyping, new style of Dorime.\n...",neutral,"na new style of hyping, new style of dorime.\n..."
1,abeg! explanation of what’s going on here is n...,neutral,abeg! explanation of what’s going on here is n...
2,"Nothing, na the new style wa dis club owners t...",neutral,"nothing, na the new style wa dis club owners t..."
3,Even people wey make the alcohol no do reach l...,negative,even people wey make the alcohol no do reach l...
4,Omg I love this the young don tire sef.,positive,omg i love this the young don tire sef.


### **Logistic Regression on Nigerian-Pidgin Language**

In [ ]:
# prompt: perform logistic regression using sentiment as dependent variable on Pidgin_Data

import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Assuming Pidgin_Data is already defined and preprocessed

# Split the data into training and testing sets
X = Pidgin_Data['Sentence ']
y = Pidgin_Data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)


In [10]:
# Split the dataset into training and testing sets
X = Pidgin_Data['Sentence ']
y = Pidgin_Data['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
log_model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.5547445255474452
Classification Report:
              precision    recall  f1-score   support

    negative       0.65      0.33      0.44        45
     neutral       0.54      0.91      0.68        65
    positive       0.40      0.07      0.12        27

    accuracy                           0.55       137
   macro avg       0.53      0.44      0.41       137
weighted avg       0.55      0.55      0.49       137



### **Using SUPPORT VECTOR Machine on Nigerian-Pidgin Dataset**

In [11]:
# Train an SVM model
svm_model = SVC()  # You can customize the kernel and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.5547445255474452
Classification Report:
              precision    recall  f1-score   support

    negative       0.65      0.33      0.44        45
     neutral       0.54      0.91      0.68        65
    positive       0.40      0.07      0.12        27

    accuracy                           0.55       137
   macro avg       0.53      0.44      0.41       137
weighted avg       0.55      0.55      0.49       137



### **Using Random Forest on Nigerian-Pidgin Dataset**

In [12]:
# Train a Random Forest model
rfc_model = RandomForestClassifier()  # You can customize the number of estimators and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.5547445255474452
Classification Report:
              precision    recall  f1-score   support

    negative       0.65      0.33      0.44        45
     neutral       0.54      0.91      0.68        65
    positive       0.40      0.07      0.12        27

    accuracy                           0.55       137
   macro avg       0.53      0.44      0.41       137
weighted avg       0.55      0.55      0.49       137



### **Using Long short-term memory on Nigerian-Pidgin Dataset**

In [13]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_test_encoded, y_pred))

Epoch 1/10
18/18 [==============================] - 11s 448ms/step - loss: 1.0055 - accuracy: 0.5531 - val_loss: 1.0207 - val_accuracy: 0.4745
Epoch 2/10
18/18 [==============================] - 10s 538ms/step - loss: 0.9382 - accuracy: 0.5568 - val_loss: 1.0150 - val_accuracy: 0.5109
Epoch 3/10
18/18 [==============================] - 8s 433ms/step - loss: 0.8509 - accuracy: 0.5971 - val_loss: 0.9897 - val_accuracy: 0.5036
Epoch 4/10
18/18 [==============================] - 9s 485ms/step - loss: 0.6855 - accuracy: 0.7674 - val_loss: 1.0465 - val_accuracy: 0.5401
Epoch 5/10
18/18 [==============================] - 10s 547ms/step - loss: 0.4314 - accuracy: 0.8681 - val_loss: 1.0659 - val_accuracy: 0.5328
Epoch 6/10
18/18 [==============================] - 7s 408ms/step - loss: 0.2686 - accuracy: 0.9359 - val_loss: 1.1781 - val_accuracy: 0.5693
Epoch 7/10
18/18 [==============================] - 10s 552ms/step - loss: 0.1424 - accuracy: 0.9707 - val_loss: 1.3431 - val_accuracy: 0.5839
Ep

### For **HAUSA**

In [19]:
Hausa_Data.head()

,Sentence,sentiment,Sentence
0,Jagoran jam'iyyar Economic Freedom Fighters wa...,neutral,jagoran jam'iyyar economic freedom fighters wa...
1,It shall not happen insha Allah,positive,it shall not happen insha allah
2,"Saniyar da ba ta da jela, Allah ke kore mata ƙ...",neutral,"saniyar da ba ta da jela, allah ke kore mata ƙ..."
3,Congratulations Brotherly....as u have taken t...,positive,congratulations brotherly....as u have taken t...
4,Alhamdullah! Alhamdullah! I am incredibly grat...,positive,alhamdullah! alhamdullah! i am incredibly grat...


### **Using Logistic Regression on Hausa Language**

In [14]:
# Split the dataset into training and testing sets
X = Hausa_Data['Sentence ']
y = Hausa_Data['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7393483709273183
Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.59      0.65        93
     neutral       0.62      0.51      0.56        99
    positive       0.79      0.92      0.85       207

    accuracy                           0.74       399
   macro avg       0.71      0.67      0.68       399
weighted avg       0.73      0.74      0.73       399



### **Using SUPPORT VECTOR Machine on Hausa Language Dataset**

In [15]:
# Train an SVM model
svm_model = SVC()  # You can customize the kernel and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7393483709273183
Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.59      0.65        93
     neutral       0.62      0.51      0.56        99
    positive       0.79      0.92      0.85       207

    accuracy                           0.74       399
   macro avg       0.71      0.67      0.68       399
weighted avg       0.73      0.74      0.73       399



### **Using Random Forest on Hausa Language Dataset**

In [16]:
# Train a Random Forest model
rfc_model = RandomForestClassifier()  # You can customize the number of estimators and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7393483709273183
Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.59      0.65        93
     neutral       0.62      0.51      0.56        99
    positive       0.79      0.92      0.85       207

    accuracy                           0.74       399
   macro avg       0.71      0.67      0.68       399
weighted avg       0.73      0.74      0.73       399



### **Using Long short-term memory on Hausa Language Dataset**

In [17]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_test_encoded, y_pred))

Epoch 1/10
50/50 [==============================] - 16s 272ms/step - loss: 0.9834 - accuracy: 0.5132 - val_loss: 0.8099 - val_accuracy: 0.5915
Epoch 2/10
50/50 [==============================] - 17s 333ms/step - loss: 0.5514 - accuracy: 0.7801 - val_loss: 0.6417 - val_accuracy: 0.7393
Epoch 3/10
50/50 [==============================] - 14s 275ms/step - loss: 0.2268 - accuracy: 0.9267 - val_loss: 0.6851 - val_accuracy: 0.7293
Epoch 4/10
50/50 [==============================] - 13s 264ms/step - loss: 0.1041 - accuracy: 0.9749 - val_loss: 0.7676 - val_accuracy: 0.7469
Epoch 5/10
50/50 [==============================] - 17s 337ms/step - loss: 0.0447 - accuracy: 0.9900 - val_loss: 0.7822 - val_accuracy: 0.7444
Epoch 6/10
50/50 [==============================] - 16s 315ms/step - loss: 0.0307 - accuracy: 0.9931 - val_loss: 0.8866 - val_accuracy: 0.7444
Epoch 7/10
50/50 [==============================] - 26s 515ms/step - loss: 0.0226 - accuracy: 0.9925 - val_loss: 0.8008 - val_accuracy: 0.7594

## For **IGBO**

In [22]:
Igbo_Data.head()

,Sentence,sentiment
0,Lol igbo people will call this one AMANSI,positive
1,Ndi b'anyi like show off too much.,negative
2,Ndoano saa nne? I stan the hardwork😂,positive
3,the kind of criminals that will have a voice i...,negative
4,"Fela Kuti, Stephen Keshi, Eche Chinonso... Lee...",neutral


### **Logistic Regression on Igbo Language dataset**

In [21]:
# Split the dataset into training and testing sets
X = Igbo_Data['Sentence ']
y = Igbo_Data['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.6190476190476191
Classification Report:
              precision    recall  f1-score   support

    negative       0.83      0.19      0.30        27
     neutral       0.61      1.00      0.76        73
    positive       0.00      0.00      0.00        26

    accuracy                           0.62       126
   macro avg       0.48      0.40      0.35       126
weighted avg       0.53      0.62      0.50       126



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using Support Vector Machine on Igbo Language Dataset**

In [23]:
# Train an SVM model
svm_model = SVC()  # You can customize the kernel and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.6190476190476191
Classification Report:
              precision    recall  f1-score   support

    negative       0.83      0.19      0.30        27
     neutral       0.61      1.00      0.76        73
    positive       0.00      0.00      0.00        26

    accuracy                           0.62       126
   macro avg       0.48      0.40      0.35       126
weighted avg       0.53      0.62      0.50       126



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using Random Forest Classifier on Igbo Language Dataset**

In [24]:
# Train a Random Forest model
rfc_model = RandomForestClassifier()  # You can customize the number of estimators and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.6190476190476191
Classification Report:
              precision    recall  f1-score   support

    negative       0.83      0.19      0.30        27
     neutral       0.61      1.00      0.76        73
    positive       0.00      0.00      0.00        26

    accuracy                           0.62       126
   macro avg       0.48      0.40      0.35       126
weighted avg       0.53      0.62      0.50       126



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using LSTM on Igbo Language Dataset**

In [25]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_test_encoded, y_pred))

Epoch 1/10
16/16 [==============================] - 6s 152ms/step - loss: 0.9666 - accuracy: 0.6208 - val_loss: 0.9764 - val_accuracy: 0.5794
Epoch 2/10
16/16 [==============================] - 2s 118ms/step - loss: 0.8602 - accuracy: 0.6387 - val_loss: 0.9584 - val_accuracy: 0.5873
Epoch 3/10
16/16 [==============================] - 2s 113ms/step - loss: 0.7469 - accuracy: 0.6707 - val_loss: 0.9643 - val_accuracy: 0.6270
Epoch 4/10
16/16 [==============================] - 2s 114ms/step - loss: 0.5177 - accuracy: 0.7824 - val_loss: 1.0696 - val_accuracy: 0.6349
Epoch 5/10
16/16 [==============================] - 3s 183ms/step - loss: 0.3768 - accuracy: 0.8723 - val_loss: 1.1365 - val_accuracy: 0.6111
Epoch 6/10
16/16 [==============================] - 3s 177ms/step - loss: 0.2633 - accuracy: 0.9441 - val_loss: 0.8880 - val_accuracy: 0.6587
Epoch 7/10
16/16 [==============================] - 2s 117ms/step - loss: 0.1373 - accuracy: 0.9860 - val_loss: 0.9307 - val_accuracy: 0.6508
Epoch 


## For **Yoruba**

In [ ]:
Yoruba_Data.head()

,Sentence,sentiment
0,koyemi o which kain hypeman be this,neutral
1,Eye services. Okan yin oni bale.,negative
2,"Yoruba parents be like ""Awon omo mi ni yi, oya...",positive
3,Ah ah oro niyen oooooo 😂😂😂😂 Osg yahoo boys in ...,neutral
4,Lol. Idan gan gan,neutral


### **Using Logistic Regression on Yoruba Language Dataset**

In [26]:
# Split the dataset into training and testing sets
X = Yoruba_Data['Sentence ']
y = Yoruba_Data['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.8918918918918919
Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.89      1.00      0.94       229
    positive       1.00      0.10      0.18        20

    accuracy                           0.89       259
   macro avg       0.63      0.37      0.37       259
weighted avg       0.87      0.89      0.85       259



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using Support Vector Machine on Yoruba Language Dataset**

In [27]:
# Train an SVM model
svm_model = SVC()  # You can customize the kernel and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.8918918918918919
Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.89      1.00      0.94       229
    positive       1.00      0.10      0.18        20

    accuracy                           0.89       259
   macro avg       0.63      0.37      0.37       259
weighted avg       0.87      0.89      0.85       259



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using Random Forest Classifier on Yoruba Language Dataset**

In [28]:
# Train a Random Forest model
rfc_model = RandomForestClassifier()  # You can customize the number of estimators and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.8918918918918919
Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.89      1.00      0.94       229
    positive       1.00      0.10      0.18        20

    accuracy                           0.89       259
   macro avg       0.63      0.37      0.37       259
weighted avg       0.87      0.89      0.85       259



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **Using LSTM on Yoruba Language Dataset**

In [29]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_test_encoded, y_pred))

Epoch 1/10
33/33 [==============================] - 13s 284ms/step - loss: 0.5903 - accuracy: 0.8645 - val_loss: 0.4061 - val_accuracy: 0.8842
Epoch 2/10
33/33 [==============================] - 7s 198ms/step - loss: 0.3570 - accuracy: 0.8858 - val_loss: 0.3184 - val_accuracy: 0.8919
Epoch 3/10
33/33 [==============================] - 8s 259ms/step - loss: 0.2393 - accuracy: 0.9003 - val_loss: 0.3076 - val_accuracy: 0.9151
Epoch 4/10
33/33 [==============================] - 6s 194ms/step - loss: 0.1558 - accuracy: 0.9409 - val_loss: 0.2974 - val_accuracy: 0.8958
Epoch 5/10
33/33 [==============================] - 10s 309ms/step - loss: 0.1145 - accuracy: 0.9593 - val_loss: 0.3037 - val_accuracy: 0.8996
Epoch 6/10
33/33 [==============================] - 12s 375ms/step - loss: 0.0758 - accuracy: 0.9710 - val_loss: 0.5148 - val_accuracy: 0.9189
Epoch 7/10
33/33 [==============================] - 13s 392ms/step - loss: 0.0506 - accuracy: 0.9894 - val_loss: 0.3353 - val_accuracy: 0.9112
Ep

# **FOR THE COMBINED DATASET**

### **Data Preprocessing**

In [38]:
# Concatenate all dataframes into one
all_languages = pd.concat([Hausa_Data, Igbo_Data, Yoruba_Data, Pidgin_Data, General_Data], ignore_index=True)


In [39]:
for sentiments in all_languages['sentiment'].unique():
  print( sentiments, (all_languages['sentiment'] == sentiments).sum())

neutral 2684
positive 1513
negative 1052


In [41]:
all_languages.head()

,What combination does the sentence have?,sentiment,Sentence,Sentence
0,English and Hausa,neutral,Jagoran jam'iyyar Economic Freedom Fighters wa...,NaN
1,English and Hausa,positive,It shall not happen insha Allah it shall not h...,NaN
2,English and Hausa,neutral,"Saniyar da ba ta da jela, Allah ke kore mata ƙ...",NaN
3,English and Hausa,positive,Congratulations Brotherly....as u have taken t...,NaN
4,English and Hausa,positive,Alhamdullah! Alhamdullah! I am incredibly grat...,NaN


In [43]:
all_languages.shape

(5249, 3)

In [42]:
#  merging 'Sentence ' and 'Sentence' columns into one called 'Sentence'
all_languages['Sentence'] = all_languages['Sentence '].fillna('') + all_languages['Sentence'].fillna('')

# Drop the original columns if you don't need them anymore
all_languages = all_languages.drop(['Sentence '], axis=1)

# Display the updated DataFrame
print(all_languages.head())

  What combination does the sentence have? sentiment  \
0                        English and Hausa   neutral   
1                        English and Hausa  positive   
2                        English and Hausa   neutral   
3                        English and Hausa  positive   
4                        English and Hausa  positive   

                                            Sentence  
0  Jagoran jam'iyyar Economic Freedom Fighters wa...  
1  It shall not happen insha Allah it shall not h...  
2  Saniyar da ba ta da jela, Allah ke kore mata ƙ...  
3  Congratulations Brotherly....as u have taken t...  
4  Alhamdullah! Alhamdullah! I am incredibly grat...  


### **LOGISTIC REGRESSION ON COMBINED-LANGUAGE DATASET**

In [44]:


# Assuming 'all_languages' DataFrame is already defined and preprocessed

# Split the dataset into training and testing sets
X = all_languages['Sentence']
y = all_languages['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_encoded, y_pred))


Epoch 1/10
132/132 [==============================] - 120s 891ms/step - loss: 0.8742 - accuracy: 0.6132 - val_loss: 0.7508 - val_accuracy: 0.6724
Epoch 2/10
132/132 [==============================] - 119s 906ms/step - loss: 0.5003 - accuracy: 0.8159 - val_loss: 0.7170 - val_accuracy: 0.7048
Epoch 3/10
132/132 [==============================] - 118s 893ms/step - loss: 0.2016 - accuracy: 0.9350 - val_loss: 0.8510 - val_accuracy: 0.7152
Epoch 4/10
132/132 [==============================] - 119s 901ms/step - loss: 0.0878 - accuracy: 0.9755 - val_loss: 1.0054 - val_accuracy: 0.7010
Epoch 5/10
132/132 [==============================] - 120s 910ms/step - loss: 0.0603 - accuracy: 0.9836 - val_loss: 1.0790 - val_accuracy: 0.6952
Epoch 6/10
132/132 [==============================] - 118s 899ms/step - loss: 0.0445 - accuracy: 0.9886 - val_loss: 1.0841 - val_accuracy: 0.7000
Epoch 7/10
132/132 [==============================] - 120s 907ms/step - loss: 0.0366 - accuracy: 0.9886 - val_loss: 1.1413 -

In [36]:
# Split the dataset into training and testing sets
X = all_languages['Sentence']
y = all_languages['sentiment'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7228571428571429
Classification Report:
              precision    recall  f1-score   support

    negative       0.72      0.45      0.55       220
     neutral       0.72      0.88      0.79       530
    positive       0.74      0.64      0.69       300

    accuracy                           0.72      1050
   macro avg       0.72      0.66      0.68      1050
weighted avg       0.72      0.72      0.71      1050



### **USING SUPPORT VECTOR CLASSIFICATION FOR COMPARATIVE ANALYSIS**

In [37]:

from sklearn.svm import SVC

# Train an SVM model
model = SVC()  # You can customize the kernel and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.7380952380952381
Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.39      0.52       220
     neutral       0.70      0.94      0.80       530
    positive       0.83      0.63      0.72       300

    accuracy                           0.74      1050
   macro avg       0.78      0.65      0.68      1050
weighted avg       0.76      0.74      0.72      1050



## **USING RANDOM FOREST CLASSIFICATION FOR COMPARATIVE ANALYSIS**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# ... (preceding code remains the same)

# Train a Random Forest model
model = RandomForestClassifier()  # You can customize the number of estimators and other parameters
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.7380952380952381
Classification Report:
              precision    recall  f1-score   support

    negative       0.85      0.37      0.51       220
     neutral       0.70      0.93      0.80       530
    positive       0.79      0.67      0.72       300

    accuracy                           0.74      1050
   macro avg       0.78      0.66      0.68      1050
weighted avg       0.76      0.74      0.72      1050



### **USING LONG SHORT-TERM MEMORY FOR COMPARATIVE ANALYSIS**

In [ ]:

# ... (preceding code remains the same)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_test_encoded = np.array([label_mapping[label] for label in y_test])

# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=10, validation_data=(X_test_padded, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Accuracy: {accuracy}")

# Generate predictions
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_test_encoded, y_pred))


Epoch 1/10
132/132 [==============================] - 103s 751ms/step - loss: 0.8699 - accuracy: 0.6129 - val_loss: 0.7773 - val_accuracy: 0.6676
Epoch 2/10
132/132 [==============================] - 102s 777ms/step - loss: 0.4804 - accuracy: 0.8117 - val_loss: 0.7301 - val_accuracy: 0.7324
Epoch 3/10
132/132 [==============================] - 100s 759ms/step - loss: 0.1811 - accuracy: 0.9433 - val_loss: 0.8375 - val_accuracy: 0.7305
Epoch 4/10
132/132 [==============================] - 101s 762ms/step - loss: 0.0938 - accuracy: 0.9733 - val_loss: 0.9523 - val_accuracy: 0.7305
Epoch 5/10
132/132 [==============================] - 103s 781ms/step - loss: 0.0522 - accuracy: 0.9857 - val_loss: 1.1143 - val_accuracy: 0.7190
Epoch 6/10
132/132 [==============================] - 101s 767ms/step - loss: 0.0365 - accuracy: 0.9893 - val_loss: 1.0978 - val_accuracy: 0.7162
Epoch 7/10
132/132 [==============================] - 102s 777ms/step - loss: 0.0353 - accuracy: 0.9895 - val_loss: 1.0665 -